In [1]:
# import libraries 
# to handle paths 
import os

# for data manipulation
import numpy as np
import pandas as pd

# for iterating
from itertools import product

In [2]:
# define directory
data_folder = f"Data"
raw_folder = f"{data_folder}/raw/experiment_2"

# create directory
preprocessed_folder = f"{data_folder}/preprocessed/experiment_2"
os.makedirs(preprocessed_folder, exist_ok=True)

In [3]:
preprocessed_folder 

'Data/preprocessed/experiment_2'

In [4]:
# define variables
algorithms = ["DiagonalCMA", "DifferentialEvolution", "PSO"]
algorithms_map = {"DiagonalCMA": "diagCMA", "DifferentialEvolution": "DE", "PSO": "PSO"}
seeds = [0, 100, 200, 300, 400]
dim = 5
suites_map = {0: "BS1", 100: "BS2", 200: "BS3", 300: "BS4", 400: "BS5"}
budgets = [10000]

# Features preparation

In [5]:
# load feature data
title = "RecombinationELAFeatureData"

features = pd.read_parquet(f"{raw_folder}/{title}.parquet"
                       , engine='pyarrow')
features.head()

,ela_distr.skewness,ela_meta.lin_simple.adjr2,ela_meta.lin_simple.intercept,ela_meta.lin_simple.coef_max,ela_meta.quad_simple.adjr2,ela_meta.quad_simple.coef.min_by_max,ela_level.lda_mmce_25,ela_level.qda_mmce_25,ela_level.lda_qda_25,ic.eps.s,ic.eps.ratio,disp.ratio_mean_02,entropy.y,entropy.sig_dth_order,f0,f1,dimension,alpha,replication
0,0.730703,0.290097,97.549523,2.313577,1.000000,0.218534,0.75001,0.853512,0.878733,1.126126,0.685686,0.144879,3.795338,0.073882,bbob_f001_i01_d02,shifted-bbob_f002_i01_d02,2,0.000000,9
1,1.170747,0.438187,154.785359,16.068023,0.999126,0.031992,0.75001,0.873101,0.859018,1.926927,1.306306,0.184309,5.260427,0.052397,bbob_f001_i01_d02,shifted-bbob_f002_i01_d02,2,0.000006,9
2,1.188080,0.441808,212.020854,29.822386,0.998976,0.017520,0.75001,0.892614,0.840240,2.207207,1.586587,0.203320,5.818897,0.048035,bbob_f001_i01_d02,shifted-bbob_f002_i01_d02,2,0.000012,9
3,1.192424,0.442625,269.256008,43.576668,0.998919,0.012184,0.75001,0.896497,0.836600,2.347347,1.746747,0.215577,6.170726,0.044085,bbob_f001_i01_d02,shifted-bbob_f002_i01_d02,2,0.000018,9
4,1.194861,0.443042,361.863879,65.831431,0.998876,0.008272,0.75001,0.900400,0.832974,2.527528,1.846847,0.230229,6.537447,0.038827,bbob_f001_i01_d02,shifted-bbob_f002_i01_d02,2,0.000028,9


In [6]:
features.shape

(1656000, 19)

In [7]:
features.dtypes

ela_distr.skewness                      float64
ela_meta.lin_simple.adjr2               float64
ela_meta.lin_simple.intercept           float64
ela_meta.lin_simple.coef_max            float64
ela_meta.quad_simple.adjr2              float64
ela_meta.quad_simple.coef.min_by_max    float64
ela_level.lda_mmce_25                   float64
ela_level.qda_mmce_25                   float64
ela_level.lda_qda_25                    float64
ic.eps.s                                float64
ic.eps.ratio                            float64
disp.ratio_mean_02                      float64
entropy.y                               float64
entropy.sig_dth_order                   float64
f0                                       object
f1                                       object
dimension                                 int64
alpha                                   float64
replication                               int64
dtype: object

In [8]:
for column in ["alpha", "dimension"]:
    print(column)
    print(features[column].unique())
    print(len(features[column].unique()))
    print("\n")

alpha
[0.00000000e+00 5.96086099e-06 1.19216864e-05 ... 9.06047679e-01
 9.30433002e-01 9.55892947e-01]
1448199


dimension
[ 2  3  5 10 20]
5




In [9]:
# filter data
features = features[features["dimension"]==dim]

features.shape

(331200, 19)

In [10]:
# drop unecessary columns:
features = features.drop(columns=["replication", "dimension"])
features.shape

(331200, 17)

In [11]:
# change data types
features['alpha'] = features['alpha'].astype(np.float32)

In [12]:
# aggregate features with median
id_columns=['f0','f1','alpha']

features=features.groupby(id_columns).median().reset_index()

features.shape

(285456, 17)

In [13]:
for column in ["alpha"]:
    print(column)
    print(len(features[column].unique()))
    print("\n")

alpha
276408




In [14]:
# check for nan
features.columns[features.isna().any()].tolist()

[]

In [15]:
features.head()

,f0,f1,alpha,ela_distr.skewness,ela_meta.lin_simple.adjr2,ela_meta.lin_simple.intercept,ela_meta.lin_simple.coef_max,ela_meta.quad_simple.adjr2,ela_meta.quad_simple.coef.min_by_max,ela_level.lda_mmce_25,ela_level.qda_mmce_25,ela_level.lda_qda_25,ic.eps.s,ic.eps.ratio,disp.ratio_mean_02,entropy.y,entropy.sig_dth_order
0,bbob_f001_i01_d05,shifted-bbob_f002_i01_d05,0.000000,0.527170,0.604210,133.970642,5.361599,1.000000,0.094300,0.809076,0.881099,0.917523,1.186186,0.665666,0.435486,4.637107,-0.060751
1,bbob_f001_i01_d05,shifted-bbob_f002_i01_d05,0.000006,1.048569,0.783936,231.883963,38.939130,0.998483,0.012985,0.832763,0.901853,0.924870,1.906907,1.066066,0.480974,5.864053,-0.019754
2,bbob_f001_i01_d05,shifted-bbob_f002_i01_d05,0.000012,1.084309,0.788637,329.796700,72.516475,0.998219,0.006973,0.812746,0.903316,0.900154,2.167167,1.286286,0.503805,6.360932,-0.014255
3,bbob_f001_i01_d05,shifted-bbob_f002_i01_d05,0.000018,1.093979,0.789486,427.708853,106.093615,0.998120,0.004766,0.794192,0.912590,0.871457,2.327327,1.426426,0.519245,6.686344,-0.010661
4,bbob_f001_i01_d05,shifted-bbob_f002_i01_d05,0.000037,1.102091,0.789855,738.798133,212.775854,0.998013,0.002377,0.768068,0.932861,0.821914,2.627628,1.726727,0.553149,7.311487,-0.007348


In [16]:
features.tail()

,f0,f1,alpha,ela_distr.skewness,ela_meta.lin_simple.adjr2,ela_meta.lin_simple.intercept,ela_meta.lin_simple.coef_max,ela_meta.quad_simple.adjr2,ela_meta.quad_simple.coef.min_by_max,ela_level.lda_mmce_25,ela_level.qda_mmce_25,ela_level.lda_qda_25,ic.eps.s,ic.eps.ratio,disp.ratio_mean_02,entropy.y,entropy.sig_dth_order
285451,bbob_f024_i01_d05,shifted-bbob_f023_i01_d05,0.860838,0.402581,0.117399,61.824134,0.936656,0.454172,0.066648,0.747559,0.757325,0.987105,1.126126,0.565566,0.687328,3.857890,-0.108619
285452,bbob_f024_i01_d05,shifted-bbob_f023_i01_d05,0.862388,0.372058,0.093797,61.117318,0.862113,0.427316,0.037614,0.746096,0.771974,0.966477,1.166166,0.585586,0.659717,3.858783,-0.099214
285453,bbob_f024_i01_d05,shifted-bbob_f023_i01_d05,0.863878,0.395387,0.088716,61.093326,0.887859,0.441916,0.074882,0.749512,0.752446,0.996101,1.126126,0.565566,0.722484,3.861477,-0.117613
285454,bbob_f024_i01_d05,shifted-bbob_f023_i01_d05,0.869615,0.368757,0.090424,59.765720,0.842494,0.412133,0.075416,0.749025,0.756355,0.990309,1.186186,0.565566,0.710929,3.835957,-0.113987
285455,bbob_f024_i01_d05,shifted-bbob_f023_i01_d05,1.000000,0.484697,0.185651,31.780423,1.585206,0.296477,0.005833,0.742187,0.746826,0.994145,1.166166,0.585586,0.879936,3.776859,-0.089021


In [17]:
features.dtypes

f0                                       object
f1                                       object
alpha                                   float64
ela_distr.skewness                      float64
ela_meta.lin_simple.adjr2               float64
ela_meta.lin_simple.intercept           float64
ela_meta.lin_simple.coef_max            float64
ela_meta.quad_simple.adjr2              float64
ela_meta.quad_simple.coef.min_by_max    float64
ela_level.lda_mmce_25                   float64
ela_level.qda_mmce_25                   float64
ela_level.lda_qda_25                    float64
ic.eps.s                                float64
ic.eps.ratio                            float64
disp.ratio_mean_02                      float64
entropy.y                               float64
entropy.sig_dth_order                   float64
dtype: object

In [18]:
# change data types
features['alpha'] = features['alpha'].astype(np.float32)

In [19]:
# save features
features.to_csv(f"{preprocessed_folder}/RecombinationELAFeatureData_preprocessed.csv", index=False)

# Merge features with selected fids as datasets

In [20]:
# to preview data
title = f"mis_final_dim_5_seed_0"
    
df = pd.read_csv(f"{raw_folder}/mis_aggregated/{title}.csv", index_col=0)
df.head()

,f0,f1,dimension,alpha
0,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05,5,0.895461
1,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05,5,0.868363
2,bbob_f003_i01_d05,shifted-bbob_f009_i01_d05,5,0.001103
3,bbob_f003_i01_d05,shifted-bbob_f023_i01_d05,5,0.785410
4,bbob_f017_i01_d05,shifted-bbob_f003_i01_d05,5,0.345955


In [21]:
df.shape

(56, 4)

In [22]:
df.dtypes

f0            object
f1            object
dimension      int64
alpha        float64
dtype: object

In [23]:
# merge features with selected fids

for seed in seeds:
    print("\n")
    print(f"seed: {seed}")
    
    # load selected fids
    title = f"mis_final_dim_{dim}_seed_{seed}"
    
    fids = pd.read_csv(f"{raw_folder}/mis_aggregated/{title}.csv", index_col=0,
                      )
    # drop dimension
    fids = fids.drop("dimension", axis=1)
    
    # change data type
    fids['alpha']=fids['alpha'].astype(np.float32)
    
    print("preview initial fids:")
    print(fids.head(1))
    print(fids.dtypes)
    print(fids.shape)
    
    # merge with features
    features_fids = pd.merge(fids, features, how='inner', on=["f0", "f1", "alpha"])
    
#     # extract f0 and f1
#     features_fids["f0"] = features_fids['f0'].str.split('_').str[1]
#     features_fids["f0"] = features_fids["f0"].replace("f(0)*", "", regex=True)
#     features_fids["f0"] = features_fids["f0"].astype(int)

#     features_fids["f1"] = features_fids['f1'].str.split('_').str[1]
#     features_fids["f1"] = features_fids["f1"].replace("f(0)*", "", regex=True)
#     features_fids["f1"] = features_fids["f1"].astype(int)
        
 
    print("\n")
    print("preview features_fids:")
    print(features_fids.head(1))
    print(features_fids.isna().sum())
    print(features_fids.dtypes)
    print(features_fids.shape)
    
    features_fids.to_csv(f"{preprocessed_folder}/suite={suites_map[seed]}_features.csv", index=False)



seed: 0
preview initial fids:
                  f0                         f1     alpha
0  bbob_f004_i01_d05  shifted-bbob_f003_i01_d05  0.895461
f0        object
f1        object
alpha    float32
dtype: object
(56, 3)


preview features_fids:
                  f0                         f1     alpha  ela_distr.skewness  \
0  bbob_f004_i01_d05  shifted-bbob_f003_i01_d05  0.895461            1.123652   

   ela_meta.lin_simple.adjr2  ela_meta.lin_simple.intercept  \
0                    0.00656                     -43.664781   

   ela_meta.lin_simple.coef_max  ela_meta.quad_simple.adjr2  \
0                      7.454908                    0.758831   

   ela_meta.quad_simple.coef.min_by_max  ela_level.lda_mmce_25  \
0                              0.018924                 0.7207   

   ela_level.qda_mmce_25  ela_level.lda_qda_25  ic.eps.s  ic.eps.ratio  \
0               0.828129              0.870275  2.447447      1.646647   

   disp.ratio_mean_02  entropy.y  entropy.sig_dth_order

# Target preparation

## optimum info

In [24]:
# read optimum info

optimum = pd.read_csv(f"{raw_folder}/{dim}D_with_opts.csv", index_col = 0)
optimum.head()

,Unnamed: 0,f0,f1,dimension,alpha,opt
0,1,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05,5,0.895461,-462.090000
1,2,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05,5,0.868363,-462.090000
2,3,bbob_f003_i01_d05,shifted-bbob_f009_i01_d05,5,0.001103,123.183831
3,4,bbob_f003_i01_d05,shifted-bbob_f023_i01_d05,5,0.785410,-361.455736
4,5,bbob_f017_i01_d05,shifted-bbob_f003_i01_d05,5,0.345955,-308.087963


In [25]:
optimum.shape

(236, 6)

In [26]:
optimum.dtypes

Unnamed: 0      int64
f0             object
f1             object
dimension       int64
alpha         float64
opt           float64
dtype: object

In [27]:
# drop
optimum = optimum.drop(["Unnamed: 0", "dimension"], axis=1)

In [28]:
# change type
optimum['alpha'] = optimum['alpha'].astype(np.float32)

In [29]:
# extract f0 and f1
optimum["f0_original"] = optimum['f0']
optimum["f0"] = optimum['f0'].str.split('_').str[1]
optimum["f0"] = optimum["f0"].replace("f(0)*", "", regex=True)
optimum["f0"] = optimum["f0"].astype(int)

optimum["f1_original"] = optimum['f1']
optimum["f1"] = optimum['f1'].str.split('_').str[1]
optimum["f1"] = optimum["f1"].replace("f(0)*", "", regex=True)
optimum["f1"] = optimum["f1"].astype(int)

In [30]:
optimum.head(10)

,f0,f1,alpha,opt,f0_original,f1_original
0,4,3,0.895461,-462.090000,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05
1,4,3,0.868363,-462.090000,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05
2,3,9,0.001103,123.183831,bbob_f003_i01_d05,shifted-bbob_f009_i01_d05
3,3,23,0.785410,-361.455736,bbob_f003_i01_d05,shifted-bbob_f023_i01_d05
4,17,3,0.345955,-308.087963,bbob_f017_i01_d05,shifted-bbob_f003_i01_d05
5,16,5,0.986449,70.258293,bbob_f016_i01_d05,shifted-bbob_f005_i01_d05
6,19,14,0.943804,-99.728959,bbob_f019_i01_d05,shifted-bbob_f014_i01_d05
7,10,22,0.999951,-54.986622,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05
8,17,3,0.276705,-338.914736,bbob_f017_i01_d05,shifted-bbob_f003_i01_d05
9,3,5,0.873739,-404.908989,bbob_f003_i01_d05,shifted-bbob_f005_i01_d05


In [31]:
optimum.dtypes

f0               int32
f1               int32
alpha          float32
opt            float64
f0_original     object
f1_original     object
dtype: object

## algorithm perfromance

In [32]:
# read files from FB (fixed budget directory)
allfiles = os.listdir(f"{raw_folder}/Affine_Notransform/csv_FB_Notransform")
allfiles

['10I1_22I1_0.999901066105531_DiagonalCMA_5D.csv',
 '10I1_22I1_0.999901066105531_DifferentialEvolution_5D.csv',
 '10I1_22I1_0.999901066105531_EMNA_5D.csv',
 '10I1_22I1_0.999901066105531_PSO_5D.csv',
 '10I1_22I1_0.999901066105531_RCobyla_5D.csv',
 '10I1_22I1_0.999950668026608_DiagonalCMA_5D.csv',
 '10I1_22I1_0.999950668026608_DifferentialEvolution_5D.csv',
 '10I1_22I1_0.999950668026608_EMNA_5D.csv',
 '10I1_22I1_0.999950668026608_PSO_5D.csv',
 '10I1_22I1_0.999950668026608_RCobyla_5D.csv',
 '10I1_5I1_1.0_DiagonalCMA_3D.csv',
 '10I1_5I1_1.0_DifferentialEvolution_3D.csv',
 '10I1_5I1_1.0_EMNA_3D.csv',
 '10I1_5I1_1.0_PSO_3D.csv',
 '10I1_5I1_1.0_RCobyla_3D.csv',
 '11I1_19I1_0.999996363512628_DiagonalCMA_5D.csv',
 '11I1_19I1_0.999996363512628_DifferentialEvolution_5D.csv',
 '11I1_19I1_0.999996363512628_EMNA_5D.csv',
 '11I1_19I1_0.999996363512628_PSO_5D.csv',
 '11I1_19I1_0.999996363512628_RCobyla_5D.csv',
 '11I1_5I1_1.0_DiagonalCMA_3D.csv',
 '11I1_5I1_1.0_DifferentialEvolution_3D.csv',
 '11I1_5I

In [33]:
# filter files for dimension = 5D and algorithm_names = ["DiagonalCMA","DifferentialEvolution", "PSO"]
files = [fname for fname in allfiles if ("DiagonalCMA_5D" in fname) or ("DifferentialEvolution_5D" in fname) or ("PSO_5D" in fname)]
files

['10I1_22I1_0.999901066105531_DiagonalCMA_5D.csv',
 '10I1_22I1_0.999901066105531_DifferentialEvolution_5D.csv',
 '10I1_22I1_0.999901066105531_PSO_5D.csv',
 '10I1_22I1_0.999950668026608_DiagonalCMA_5D.csv',
 '10I1_22I1_0.999950668026608_DifferentialEvolution_5D.csv',
 '10I1_22I1_0.999950668026608_PSO_5D.csv',
 '11I1_19I1_0.999996363512628_DiagonalCMA_5D.csv',
 '11I1_19I1_0.999996363512628_DifferentialEvolution_5D.csv',
 '11I1_19I1_0.999996363512628_PSO_5D.csv',
 '12I1_22I1_0.999998591270096_DiagonalCMA_5D.csv',
 '12I1_22I1_0.999998591270096_DifferentialEvolution_5D.csv',
 '12I1_22I1_0.999998591270096_PSO_5D.csv',
 '12I1_9I1_0.644579089078396_DiagonalCMA_5D.csv',
 '12I1_9I1_0.644579089078396_DifferentialEvolution_5D.csv',
 '12I1_9I1_0.644579089078396_PSO_5D.csv',
 '13I1_5I1_0.994414616101147_DiagonalCMA_5D.csv',
 '13I1_5I1_0.994414616101147_DifferentialEvolution_5D.csv',
 '13I1_5I1_0.994414616101147_PSO_5D.csv',
 '13I1_5I1_0.994503172328524_DiagonalCMA_5D.csv',
 '13I1_5I1_0.9945031723285

In [34]:
# preview target data

df = pd.read_csv(f"{raw_folder}/Affine_Notransform/csv_FB_Notransform/{files[0]}", index_col=0)
df.head(7)

,DIM,funcId,ID,runtime,run,f(x)
1,5,28,DiagonalCMA,100,1,107109.734909
2,5,28,DiagonalCMA,250,1,2024.303683
3,5,28,DiagonalCMA,500,1,1882.289720
4,5,28,DiagonalCMA,1000,1,461.843198
5,5,28,DiagonalCMA,2000,1,24.808831
6,5,28,DiagonalCMA,5000,1,-47.404046
7,5,28,DiagonalCMA,10000,1,-48.472019


In [35]:
df.shape

(175, 6)

In [36]:
df.dtypes

DIM          int64
funcId       int64
ID          object
runtime      int64
run          int64
f(x)       float64
dtype: object

In [37]:
# create a placeholder
target = pd.DataFrame()

for filename in files: 
    # load target data
    target_temp = pd.read_csv(f"{raw_folder}/Affine_Notransform/csv_FB_Notransform/{filename}", index_col=0)
    
    # preview data
    print(target_temp.head())
    print(target_temp.shape)
    
    for column in ["funcId", "ID", "DIM", "runtime", "run"]:
        print(column)
        print(target_temp[column].unique())
    
    # add fids
    target_temp["ID"] = filename.split(".csv")[0]
    target_temp[["f0","f1", "alpha", "optimizer_name", "dimension"]] = target_temp['ID'].str.split('_', expand=True)
    target_temp['f0'] = target_temp['f0'].str.split('I').str[0].astype(int)
    target_temp['f1'] = target_temp['f1'].str.split('I').str[0].astype(int)
    
    # change datatype
    target_temp['alpha'] = target_temp['alpha'].astype(np.float32)
   
    # rename columns
    target_temp = target_temp.rename(columns={"runtime": "budget"})
    
    # drop 
    target_temp = target_temp.drop(["ID", "DIM", "dimension"], axis=1)
    print(target_temp.head(7))
    print(target_temp.dtypes)
    print(target_temp.shape)
    
    # concat
    target = pd.concat([target, target_temp], axis=0)

   DIM  funcId           ID  runtime  run           f(x)
1    5      28  DiagonalCMA      100    1  107109.734909
2    5      28  DiagonalCMA      250    1    2024.303683
3    5      28  DiagonalCMA      500    1    1882.289720
4    5      28  DiagonalCMA     1000    1     461.843198
5    5      28  DiagonalCMA     2000    1      24.808831
(175, 6)
funcId
[28]
ID
['DiagonalCMA']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run           f(x)  f0  f1     alpha optimizer_name
1      28     100    1  107109.734909  10  22  0.999901    DiagonalCMA
2      28     250    1    2024.303683  10  22  0.999901    DiagonalCMA
3      28     500    1    1882.289720  10  22  0.999901    DiagonalCMA
4      28    1000    1     461.843198  10  22  0.999901    DiagonalCMA
5      28    2000    1      24.808831  10  22  0.999901    DiagonalCMA
6      28    5000    1     -47.404046  10  22  0.9

   DIM  funcId                     ID  runtime  run        f(x)
1    5      26  DifferentialEvolution      100    1  465.608918
2    5      26  DifferentialEvolution      250    1  180.304184
3    5      26  DifferentialEvolution      500    1   79.010968
4    5      26  DifferentialEvolution     1000    1   52.729070
5    5      26  DifferentialEvolution     2000    1   43.435480
(175, 6)
funcId
[26]
ID
['DifferentialEvolution']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run        f(x)  f0  f1     alpha         optimizer_name
1      26     100    1  465.608918  13   5  0.999117  DifferentialEvolution
2      26     250    1  180.304184  13   5  0.999117  DifferentialEvolution
3      26     500    1   79.010968  13   5  0.999117  DifferentialEvolution
4      26    1000    1   52.729070  13   5  0.999117  DifferentialEvolution
5      26    2000    1   43.435480  13   5  

   DIM  funcId   ID  runtime  run       f(x)
1    5      25  PSO      100    1  65.736300
2    5      25  PSO      250    1  65.056346
3    5      25  PSO      500    1  58.919706
4    5      25  PSO     1000    1  58.919706
5    5      25  PSO     2000    1  58.919706
(175, 6)
funcId
[25]
ID
['PSO']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run       f(x)  f0  f1    alpha optimizer_name
1      25     100    1  65.736300  16  19  0.82649            PSO
2      25     250    1  65.056346  16  19  0.82649            PSO
3      25     500    1  58.919706  16  19  0.82649            PSO
4      25    1000    1  58.919706  16  19  0.82649            PSO
5      25    2000    1  58.919706  16  19  0.82649            PSO
6      25    5000    1  58.919706  16  19  0.82649            PSO
7      25   10000    1  58.919706  16  19  0.82649            PSO
funcId              int64
bu

   DIM  funcId           ID  runtime  run        f(x)
1    5      27  DiagonalCMA      100    1 -834.615636
2    5      27  DiagonalCMA      250    1 -837.304412
3    5      27  DiagonalCMA      500    1 -837.932110
4    5      27  DiagonalCMA     1000    1 -838.108413
5    5      27  DiagonalCMA     2000    1 -838.112555
(175, 6)
funcId
[27]
ID
['DiagonalCMA']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run        f(x)  f0  f1     alpha optimizer_name
1      27     100    1 -834.615636  16  22  0.148178    DiagonalCMA
2      27     250    1 -837.304412  16  22  0.148178    DiagonalCMA
3      27     500    1 -837.932110  16  22  0.148178    DiagonalCMA
4      27    1000    1 -838.108413  16  22  0.148178    DiagonalCMA
5      27    2000    1 -838.112555  16  22  0.148178    DiagonalCMA
6      27    5000    1 -838.112577  16  22  0.148178    DiagonalCMA
7      27   10000 

   DIM  funcId           ID  runtime  run       f(x)
1    5      26  DiagonalCMA      100    1  74.624747
2    5      26  DiagonalCMA      250    1  72.123631
3    5      26  DiagonalCMA      500    1  70.660311
4    5      26  DiagonalCMA     1000    1  70.412223
5    5      26  DiagonalCMA     2000    1  70.393175
(175, 6)
funcId
[26]
ID
['DiagonalCMA']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run       f(x)  f0  f1     alpha optimizer_name
1      26     100    1  74.624747  16   5  0.986449    DiagonalCMA
2      26     250    1  72.123631  16   5  0.986449    DiagonalCMA
3      26     500    1  70.660311  16   5  0.986449    DiagonalCMA
4      26    1000    1  70.412223  16   5  0.986449    DiagonalCMA
5      26    2000    1  70.393175  16   5  0.986449    DiagonalCMA
6      26    5000    1  70.314516  16   5  0.986449    DiagonalCMA
7      26   10000    1  70.2982

...

   funcId  budget  run        f(x)  f0  f1     alpha optimizer_name
1      25     100    1 -327.619562   3   4  0.041165            PSO
2      25     250    1 -327.619562   3   4  0.041165            PSO
3      25     500    1 -327.619562   3   4  0.041165            PSO
4      25    1000    1 -327.619562   3   4  0.041165            PSO
5      25    2000    1 -327.619562   3   4  0.041165            PSO
6      25    5000    1 -327.619562   3   4  0.041165            PSO
7      25   10000    1 -327.619562   3   4  0.041165            PSO
funcId              int64
budget              int64
run                 int64
f(x)              float64
f0                  int32
f1                  int32
alpha             float32
optimizer_name     object
dtype: object
(175, 8)
   DIM  funcId           ID  runtime  run        f(x)
1    5      27  DiagonalCMA      100    1 -334.288855
2    5      27  DiagonalCMA      250    1 -346.174388
3    5      27  DiagonalCMA      500    1 -347.170240
4    5   

   DIM  funcId           ID  runtime  run        f(x)
1    5      25  DiagonalCMA      100    1 -352.583074
2    5      25  DiagonalCMA      250    1 -398.351408
3    5      25  DiagonalCMA      500    1 -399.034056
4    5      25  DiagonalCMA     1000    1 -399.581241
5    5      25  DiagonalCMA     2000    1 -409.370613
(175, 6)
funcId
[25]
ID
['DiagonalCMA']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run        f(x)  f0  f1     alpha optimizer_name
1      25     100    1 -352.583074   3   5  0.896192    DiagonalCMA
2      25     250    1 -398.351408   3   5  0.896192    DiagonalCMA
3      25     500    1 -399.034056   3   5  0.896192    DiagonalCMA
4      25    1000    1 -399.581241   3   5  0.896192    DiagonalCMA
5      25    2000    1 -409.370613   3   5  0.896192    DiagonalCMA
6      25    5000    1 -409.383164   3   5  0.896192    DiagonalCMA
7      25   10000 

   DIM  funcId           ID  runtime  run       f(x)
1    5      26  DiagonalCMA      100    1  79.767849
2    5      26  DiagonalCMA      250    1  39.503612
3    5      26  DiagonalCMA      500    1  35.764789
4    5      26  DiagonalCMA     1000    1  35.700021
5    5      26  DiagonalCMA     2000    1  35.699810
(175, 6)
funcId
[26]
ID
['DiagonalCMA']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run       f(x)  f0  f1     alpha optimizer_name
1      26     100    1  79.767849   3   6  0.000402    DiagonalCMA
2      26     250    1  39.503612   3   6  0.000402    DiagonalCMA
3      26     500    1  35.764789   3   6  0.000402    DiagonalCMA
4      26    1000    1  35.700021   3   6  0.000402    DiagonalCMA
5      26    2000    1  35.699810   3   6  0.000402    DiagonalCMA
6      26    5000    1  35.699810   3   6  0.000402    DiagonalCMA
7      26   10000    1  35.6998

   DIM  funcId           ID  runtime  run        f(x)
1    5      25  DiagonalCMA      100    1 -396.336081
2    5      25  DiagonalCMA      250    1 -428.889467
3    5      25  DiagonalCMA      500    1 -430.128305
4    5      25  DiagonalCMA     1000    1 -430.251406
5    5      25  DiagonalCMA     2000    1 -436.221099
(175, 6)
funcId
[25]
ID
['DiagonalCMA']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run        f(x)  f0  f1     alpha optimizer_name
1      25     100    1 -396.336081   4   3  0.835679    DiagonalCMA
2      25     250    1 -428.889467   4   3  0.835679    DiagonalCMA
3      25     500    1 -430.128305   4   3  0.835679    DiagonalCMA
4      25    1000    1 -430.251406   4   3  0.835679    DiagonalCMA
5      25    2000    1 -436.221099   4   3  0.835679    DiagonalCMA
6      25    5000    1 -446.170690   4   3  0.835679    DiagonalCMA
7      25   10000 

   DIM  funcId                     ID  runtime  run        f(x)
1    5      26  DifferentialEvolution      100    1  529.174696
2    5      26  DifferentialEvolution      250    1  151.788747
3    5      26  DifferentialEvolution      500    1  140.360061
4    5      26  DifferentialEvolution     1000    1  128.475180
5    5      26  DifferentialEvolution     2000    1  126.242860
(175, 6)
funcId
[26]
ID
['DifferentialEvolution']
DIM
[5]
runtime
[  100   250   500  1000  2000  5000 10000]
run
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
   funcId  budget  run        f(x)  f0  f1  alpha         optimizer_name
1      26     100    1  529.174696   9  14    1.0  DifferentialEvolution
2      26     250    1  151.788747   9  14    1.0  DifferentialEvolution
3      26     500    1  140.360061   9  14    1.0  DifferentialEvolution
4      26    1000    1  128.475180   9  14    1.0  DifferentialEvolution
5      26    2000    1  126.242860   9  14    1.0  Differen

In [38]:
target.head()

,funcId,budget,run,f(x),f0,f1,alpha,optimizer_name
1,28,100,1,107109.734909,10,22,0.999901,DiagonalCMA
2,28,250,1,2024.303683,10,22,0.999901,DiagonalCMA
3,28,500,1,1882.289720,10,22,0.999901,DiagonalCMA
4,28,1000,1,461.843198,10,22,0.999901,DiagonalCMA
5,28,2000,1,24.808831,10,22,0.999901,DiagonalCMA


In [39]:
target.shape

(123900, 8)

In [40]:
target.dtypes

funcId              int64
budget              int64
run                 int64
f(x)              float64
f0                  int32
f1                  int32
alpha             float32
optimizer_name     object
dtype: object

In [41]:
optimum.head()

,f0,f1,alpha,opt,f0_original,f1_original
0,4,3,0.895461,-462.090000,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05
1,4,3,0.868363,-462.090000,bbob_f004_i01_d05,shifted-bbob_f003_i01_d05
2,3,9,0.001103,123.183831,bbob_f003_i01_d05,shifted-bbob_f009_i01_d05
3,3,23,0.785410,-361.455736,bbob_f003_i01_d05,shifted-bbob_f023_i01_d05
4,17,3,0.345955,-308.087963,bbob_f017_i01_d05,shifted-bbob_f003_i01_d05


In [42]:
optimum.dtypes

f0               int32
f1               int32
alpha          float32
opt            float64
f0_original     object
f1_original     object
dtype: object

# Merge optimum and solutions

In [43]:
target = pd.merge(target, optimum, how='inner', on=["f0", "f1", "alpha"])
target.head()

,funcId,budget,run,f(x),f0,f1,alpha,optimizer_name,opt,f0_original,f1_original
0,28,100,1,107109.734909,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05
1,28,250,1,2024.303683,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05
2,28,500,1,1882.289720,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05
3,28,1000,1,461.843198,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05
4,28,2000,1,24.808831,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05


In [44]:
target.shape

(123900, 11)

In [45]:
target.dtypes

funcId              int64
budget              int64
run                 int64
f(x)              float64
f0                  int32
f1                  int32
alpha             float32
optimizer_name     object
opt               float64
f0_original        object
f1_original        object
dtype: object

In [46]:
# calculate precision 
target["precision"] = abs(target["f(x)"] - target["opt"])
target.head()

,funcId,budget,run,f(x),f0,f1,alpha,optimizer_name,opt,f0_original,f1_original,precision
0,28,100,1,107109.734909,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05,107164.768407
1,28,250,1,2024.303683,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05,2079.337181
2,28,500,1,1882.289720,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05,1937.323219
3,28,1000,1,461.843198,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05,516.876697
4,28,2000,1,24.808831,10,22,0.999901,DiagonalCMA,-55.033498,bbob_f010_i01_d05,shifted-bbob_f022_i01_d05,79.842330


In [47]:
# group by budget with median
target = target.groupby(["optimizer_name", "budget", "f0_original", "f1_original", "alpha"])["precision"].median().reset_index()
target.head()

,optimizer_name,budget,f0_original,f1_original,alpha,precision
0,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f003_i01_d05,0.701592,10.260378
1,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.724414,2.147590
2,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.792491,1.587453
3,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.803477,1.773336
4,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.807921,1.784482


In [48]:
target.shape

(4956, 6)

In [49]:
target.dtypes

optimizer_name     object
budget              int64
f0_original        object
f1_original        object
alpha             float64
precision         float64
dtype: object

In [50]:
target['alpha'] = target['alpha'].astype(np.float32)

In [51]:
target.shape

(4956, 6)

In [52]:
target.isna().sum()

optimizer_name    0
budget            0
f0_original       0
f1_original       0
alpha             0
precision         0
dtype: int64

In [53]:
target = target.rename(columns={"f0_original": "f0", "f1_original": "f1"})

In [54]:
# save
target.to_csv(f"{preprocessed_folder}/target_affine.csv", index=False)

In [55]:
target

,optimizer_name,budget,f0,f1,alpha,precision
0,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f003_i01_d05,0.701592,10.260378
1,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.724414,2.147590
2,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.792491,1.587453
3,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.803477,1.773336
4,DiagonalCMA,100,bbob_f001_i01_d05,shifted-bbob_f019_i01_d05,0.807921,1.784482
...,...,...,...,...,...,...
4951,PSO,10000,bbob_f023_i01_d05,shifted-bbob_f015_i01_d05,0.019712,61.100677
4952,PSO,10000,bbob_f023_i01_d05,shifted-bbob_f017_i01_d05,0.956284,2.893805
4953,PSO,10000,bbob_f024_i01_d05,shifted-bbob_f014_i01_d05,0.996954,45.708286
4954,PSO,10000,bbob_f024_i01_d05,shifted-bbob_f019_i01_d05,0.839298,37.238557


In [56]:
target.dtypes

optimizer_name     object
budget              int64
f0                 object
f1                 object
alpha             float32
precision         float64
dtype: object

In [57]:
# add fids
# target_temp["ID"] = filename.split(".csv")[0]
# target_temp[["f0","f1", "alpha", "optimizer_name", "dimension"]] = target_temp['ID'].str.split('_', expand=True)
# target_temp['f0'] = target_temp['f0'].str.split('I').str[0].astype(int)
# target_temp['f1'] = target_temp['f1'].str.split('I').str[0].astype(int)
# target_temp['dimension'] = target_temp['dimension'].str.split('D').str[0].astype(int)

# Merge features and target

In [58]:
for algorithm_name, budget in product(algorithms, budgets):
    print(f"algorithm_name: {algorithm_name}, budget: {budget}")
    
    # select budget
    target_temp = target[(target["budget"]==budget)&(target["optimizer_name"]==algorithm_name)]
#     print(target_temp.head(1))
    print(target_temp.shape)
#     print(target_temp.dtypes)
    
    for seed in seeds:
#         print("\n")
        print(f"seed: {seed}")
        
        # load features
        features = pd.read_csv(f"{preprocessed_folder}/suite={suites_map[seed]}_features.csv")
        features['alpha'] = features['alpha'].astype(np.float32)

#         print(features.head(1))
#         print(features.dtypes)
        print(features.shape)
        
        # merge on "f0", "f1", "alpha", "dimension"
        dataset = pd.merge(features, target_temp, how='inner', on=["f0", "f1", "alpha"])
        
        # drop unecessary columns
        dataset = dataset.drop(["optimizer_name", "budget"], axis=1)
#         print("\n")
#         print(dataset.head(1))
        # print(dataset.isna().sum())
        # print(dataset.dtypes)
        print(dataset.shape)
        
        dataset.to_csv(f"{preprocessed_folder}/suite={suites_map[seed]}_algorithm_name={algorithms_map[algorithm_name]}_budget={budget}_dataset.csv", index=False)

algorithm_name: DiagonalCMA, budget: 10000
(236, 6)
seed: 0
(56, 17)
(56, 18)
seed: 100
(57, 17)
(57, 18)
seed: 200
(56, 17)
(56, 18)
seed: 300
(55, 17)
(55, 18)
seed: 400
(53, 17)
(53, 18)
algorithm_name: DifferentialEvolution, budget: 10000
(236, 6)
seed: 0
(56, 17)
(56, 18)
seed: 100
(57, 17)
(57, 18)
seed: 200
(56, 17)
(56, 18)
seed: 300
(55, 17)
(55, 18)
seed: 400
(53, 17)
(53, 18)
algorithm_name: PSO, budget: 10000
(236, 6)
seed: 0
(56, 17)
(56, 18)
seed: 100
(57, 17)
(57, 18)
seed: 200
(56, 17)
(56, 18)
seed: 300
(55, 17)
(55, 18)
seed: 400
(53, 17)
(53, 18)


In [59]:
dataset.head()

,f0,f1,alpha,ela_distr.skewness,ela_meta.lin_simple.adjr2,ela_meta.lin_simple.intercept,ela_meta.lin_simple.coef_max,ela_meta.quad_simple.adjr2,ela_meta.quad_simple.coef.min_by_max,ela_level.lda_mmce_25,ela_level.qda_mmce_25,ela_level.lda_qda_25,ic.eps.s,ic.eps.ratio,disp.ratio_mean_02,entropy.y,entropy.sig_dth_order,precision
0,bbob_f022_i01_d05,shifted-bbob_f001_i01_d05,0.848012,0.489882,0.585609,-27.477388,4.561264,0.993950,0.086547,0.802254,0.874518,0.917367,1.106106,0.585586,0.452977,4.493945,-0.077192,15.601375
1,bbob_f016_i01_d05,shifted-bbob_f003_i01_d05,0.783305,1.375567,0.627696,235.023191,135.253495,0.949710,0.004508,0.790523,0.890135,0.888093,2.647648,1.786787,0.521974,7.195247,-0.043483,44.767704
2,bbob_f003_i01_d05,shifted-bbob_f016_i01_d05,0.423922,0.619294,0.575229,-7.465732,22.895165,0.932491,0.038780,0.775389,0.857407,0.904342,1.806807,1.266266,0.573222,5.890993,-0.046929,52.526503
3,bbob_f016_i01_d05,shifted-bbob_f014_i01_d05,0.994975,3.031577,0.236427,17.936361,11.385385,0.413396,0.045042,0.729983,0.887206,0.822788,1.926927,0.785786,0.484004,5.144421,-0.147046,5.650618
4,bbob_f019_i01_d05,shifted-bbob_f023_i01_d05,0.526235,1.802551,0.036193,-5.481839,1.283864,0.559121,0.085689,0.750001,0.788094,0.951664,1.386386,0.605606,0.620144,4.307913,-0.139142,5.769789


In [60]:
dataset.shape

(53, 18)

In [61]:
dataset.dtypes

f0                                       object
f1                                       object
alpha                                   float32
ela_distr.skewness                      float64
ela_meta.lin_simple.adjr2               float64
ela_meta.lin_simple.intercept           float64
ela_meta.lin_simple.coef_max            float64
ela_meta.quad_simple.adjr2              float64
ela_meta.quad_simple.coef.min_by_max    float64
ela_level.lda_mmce_25                   float64
ela_level.qda_mmce_25                   float64
ela_level.lda_qda_25                    float64
ic.eps.s                                float64
ic.eps.ratio                            float64
disp.ratio_mean_02                      float64
entropy.y                               float64
entropy.sig_dth_order                   float64
precision                               float64
dtype: object